In [ ]:
import torch
import numpy as np

# Linear Regression

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [ ]:
inputs = torch.tensor(inputs)
targets = torch.tensor(targets)
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [ ]:
def model(x):
  y = x @ w.t() + b
  return y

In [ ]:
preds = model(inputs)

In [ ]:
def mse(t1,t2):
  diff = t1 - t2
  loss = torch.sum(diff * diff)/diff.numel()
  return loss

In [ ]:
loss = mse(preds,targets)
print(loss)

tensor(15766.3701, grad_fn=<DivBackward0>)


# Compute Gradients

In [ ]:
loss.backward()

In [ ]:
w.grad

tensor([[13632.9199, 13279.4307,  8377.1846],
        [ 1769.6460,  -882.2419,   -37.8505]])

In [ ]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

In [ ]:
print(loss)

tensor(15766.3701, grad_fn=<DivBackward0>)


In [ ]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [ ]:
preds = model(inputs)

In [ ]:
loss = mse(preds,targets)

In [ ]:
loss.backward()

In [ ]:
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5

In [ ]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [ ]:
preds = model(inputs)

In [ ]:
loss = mse(preds,targets)
print(loss)

tensor(9096.0957, grad_fn=<DivBackward0>)


# Run multiple epochs

In [ ]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
loss

tensor(1097.4160, grad_fn=<DivBackward0>)

# Linear regression using PyTorch built-ins

In [ ]:
import  torch.nn as nn

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [ ]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
model = nn.Linear(3,2)

In [ ]:
print(model.weight)
print(model.bias)
print(list(model.parameters()))

Parameter containing:
tensor([[-0.2036,  0.2225, -0.3058],
        [ 0.3488, -0.1233,  0.0975]], requires_grad=True)
Parameter containing:
tensor([ 0.1327, -0.0411], requires_grad=True)
[Parameter containing:
tensor([[-0.2036,  0.2225, -0.3058],
        [ 0.3488, -0.1233,  0.0975]], requires_grad=True), Parameter containing:
tensor([ 0.1327, -0.0411], requires_grad=True)]


In [ ]:
import torch.nn.functional as F

In [ ]:
loss_fn = F.mse_loss

In [ ]:
preds = model(inputs)
loss_fn(preds,targets)

tensor(7634.1597, grad_fn=<MseLossBackward>)

# Optimizers

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 31.3845
Epoch [20/100], Loss: 37.6321
Epoch [30/100], Loss: 25.8685
Epoch [40/100], Loss: 16.6188
Epoch [50/100], Loss: 15.1828
Epoch [60/100], Loss: 15.0144
Epoch [70/100], Loss: 13.6694
Epoch [80/100], Loss: 18.1131
Epoch [90/100], Loss: 10.7933
Epoch [100/100], Loss: 15.0694


In [ ]:
preds = model(inputs)

In [ ]:
preds

tensor([[ 57.3136,  70.7569],
        [ 79.2699,  98.6797],
        [123.4650, 136.2264],
        [ 22.8318,  39.8565],
        [ 95.9371, 114.0477],
        [ 56.0253,  69.6695],
        [ 78.6271,  98.4271],
        [123.4888, 136.6464],
        [ 24.1201,  40.9440],
        [ 96.5827, 114.8825],
        [ 56.6709,  70.5043],
        [ 77.9816,  97.5923],
        [124.1077, 136.4790],
        [ 22.1862,  39.0217],
        [ 97.2254, 115.1352]], grad_fn=<AddmmBackward>)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])